In [12]:
import time
import datetime
import pandas as pd
import plotly.subplots # import make_subplots
import plotly.offline as offline
import plotly.graph_objs as graph_obj

start = time.time()


class StockGraphParser:

    __base_url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'

    @classmethod
    def initialize(cls):
        return

    @classmethod
    def __make_url(cls, code):
        return cls.__base_url.format(code=code)

    @classmethod
    def get_data_frame(cls, code):
        data_frame = pd.DataFrame()
        url = cls.__make_url(code)

        for page in range(1, 30):
            page_url = '{url}&page={page}'.format(url = url, page = page)
            data_frame = data_frame.append(pd.read_html(page_url, header = 0)[0], ignore_index = True)
            print(page)

        data_frame = data_frame.dropna()
            
        # Rename columns
        data_frame = data_frame.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})

        # Convert data type
        data_frame[['close', 'diff', 'open', 'high', 'low', 'volume']] = data_frame[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
        data_frame['date'] = pd.to_datetime(data_frame['date'])

        # Sort by date
        data_frame = data_frame.sort_values(by = ['date'], ascending=True)

        
        # Gem SMA
        data_frame['sma5'] = data_frame['close'].rolling(5).mean()
        data_frame['sma20'] = data_frame['close'].rolling(20).mean()
        data_frame['sma100'] = data_frame['close'].rolling(100).mean()
        data_frame['sma200'] = data_frame['close'].rolling(200).mean()
        
        # Gen EMA
        data_frame['ema5'] = data_frame['close'].ewm(5).mean()
        data_frame['ema20'] = data_frame['close'].ewm(20).mean()
        data_frame['ema100'] =data_frame['close'].ewm(100).mean()
        data_frame['ema200'] = data_frame['close'].ewm(200).mean()
                    
        return data_frame

    
data_frame = StockGraphParser.get_data_frame('002700')


# Get empty date
dt_all = pd.date_range(start=data_frame['date'].iloc[0],end=data_frame['date'].iloc[-1])
dt_obs = [d.strftime("%Y-%m-%d") for d in data_frame['date']]
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
print(data_frame)

item_name = '신일전자'

moving_avg = ['sma5','sma20','sma100','sma200','ema5','ema20','ema100','ema200']

# GRAPH PART
# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True)

fig = make_subplots(
    rows=3, 
    cols=1, 
    shared_xaxes = True,
    specs=[[{"rowspan" : 2}],
           [None],
           [{}]])



fig.add_trace(graph_obj.Candlestick(
                          x=data_frame.date,
                          open = data_frame.open,
                          high = data_frame.high,
                          low = data_frame.low,
                          close = data_frame.close,
    
                          increasing_line_color= 'red', 
                          decreasing_line_color= 'blue',
                          # y=data_frame.close,
                          name=item_name),
              row=1, col=1)


for avg in moving_avg:
    fig.add_trace(graph_obj.Scatter(
                                    x = data_frame.date,
                                    y = data_frame[avg]),
                  row=1, col=1)

fig.add_trace(graph_obj.Bar(x = data_frame.date,
                            y = data_frame.volume,
                            name = '거래량'),
              row=3, col=1)


#fig.update_layout(height = 600)
fig.update_xaxes(rangeslider_visible = False)

fig.update_xaxes(
    rangebreaks=[dict(values=dt_breaks)] # hide dates with no values
)

fig.show()

print("Get - Show :", time.time() - start) 




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
          date  close  diff  open  high   low    volume    sma5    sma20  \
433 2019-06-10   2590    65  2545  2645  2525  12072488     NaN      NaN   
432 2019-06-11   2540    50  2610  2615  2515   4982415     NaN      NaN   
431 2019-06-12   2400   140  2520  2520  2370   8067703     NaN      NaN   
430 2019-06-13   2420    20  2390  2430  2360   3401211     NaN      NaN   
429 2019-06-14   2250   170  2420  2425  2240   7155555  2440.0      NaN   
..         ...    ...   ...   ...   ...   ...       ...     ...      ...   
5   2020-07-31   1990    25  2025  2035  1980   3220606  1955.0  1899.75   
4   2020-08-03   1915    75  1985  1985  1900   2559743  1958.0  1896.00   
3   2020-08-04   1925    10  1905  1950  1880   2000325  1956.0  1893.00   
2   2020-08-05   1900    25  1910  1940  1900   1739766  1949.0  1889.00   
1   2020-08-06   1900     0  1900  1920  1885   1567844  1926.0  1886.50   

      sma

Get - Show : 18.13183307647705
